In [1]:
!nvidia-smi

Thu Mar 10 23:15:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:07:00.0  On |                  N/A |
|  0%   48C    P8    16W / 170W |    367MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# dump gpu memory
#torch.cuda.empty_cache()

In [1]:
z_dim = 128
n_classes = 1000

import matplotlib.pyplot as plt
import numpy as np

import os 
import sys

sys.path.insert(1, os.path.join(sys.path[0], '..'))

# download stuff for one hot encoding of y categories

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# load big gan model

import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample, display_in_terminal)
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load pre-trained model tokenizer (vocabulary)
model = BigGAN.from_pretrained('../pretrained/biggan-deep-512')#model.to('cuda')
model.to('cuda')

[nltk_data] Downloading package wordnet to /home/ms/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ms/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
INFO:pytorch_pretrained_biggan.model:loading model ../pretrained/biggan-deep-512 from cache at ../pretrained/biggan-deep-512/pytorch_model.bin
INFO:pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
    

BigGAN(
  (embeddings): Linear(in_features=1000, out_features=128, bias=False)
  (generator): Generator(
    (gen_z): Linear(in_features=256, out_features=32768, bias=True)
    (layers): ModuleList(
      (0): GenBlock(
        (bn_0): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=2048, bias=False)
          (offset): Linear(in_features=256, out_features=2048, bias=False)
        )
        (conv_0): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))
        (bn_1): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn_2): BigGANBatchNorm(
          (scale): Linear(in_features=256, out_features=512, bias=False)
          (offset): Linear(in_features=256, out_features=512, bias=False)
        )
        (conv_2): Conv2d(512, 512, kernel_size=(3, 

In [ ]:
# basic image generation

# Prepare a input
# truncation = 0.4
# class_vector = one_hot_from_names(['Blenheim spaniel'], batch_size=5)
# noise_vector = truncated_noise_sample(truncation=truncation, batch_size=5, seed=50)

# # All in tensors
# noise_vector = torch.from_numpy(noise_vector)
# class_vector = torch.from_numpy(class_vector)

# # If you have a GPU, put everything on cuda
# noise_vector = noise_vector.to('cuda')
# class_vector = class_vector.to('cuda')
# noise_vector.shape

# # Generate an image
# with torch.no_grad():
#     output = model(noise_vector, class_vector, truncation)

In [ ]:
# visualize z sample

dim_z = 128
y = np.linspace(0, dim_z, dim_z)
truncation = 1.
z = truncated_noise_sample(truncation=truncation, batch_size=1, seed=50)
plt.scatter(y,z)
plt.show()

In [ ]:
# If you have a GPU put back on CPU
#output = output.to('cpu')

# If you have a sixtel compatible terminal you can display the images in the terminal
# (see https://github.com/saitoha/libsixel for details)
#display_in_terminal(output)

# Save results as png images
#save_as_images(output)

In [4]:
# interpolation funcs

def interpolate_linear(v1, v2, num_steps):
  vectors = []
  for x in np.linspace(0.0, 1.0, num_steps):
    vectors.append(v2*x+v1*(1-x))  
  return np.array(vectors)


def interpolate_hypersphere(v1, v2, num_steps):

  v1_norm = np.linalg.norm(v1)
  v2_norm = np.linalg.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
    interpolated_norm =  np.linalg.norm(interpolated)
    interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return np.array(vectors)

# im funcs

def convert_to_images(obj):
    """ Convert an output tensor from BigGAN in a list of images.
        Params:
            obj: tensor or numpy array of shape (batch_size, channels, height, width)
        Output:
            list of Pillow Images of size (height, width)
    """
    try:
        from PIL import Image
    except ImportError:
        raise ImportError("Please install Pillow to use images: pip install Pillow")

    if not isinstance(obj, np.ndarray):
        obj = obj.cpu().detach().numpy()

    obj = obj.transpose((0, 2, 3, 1))
    obj = np.clip(((obj + 1) / 2.0) * 256, 0, 255)

    img = []
    for i, out in enumerate(obj):
        out_array = np.asarray(np.uint8(out), dtype=np.uint8)
        img.append(Image.fromarray(out_array))
    return img


def save_as_images(obj, file_name='output'):
    """ Convert and save an output tensor from BigGAN in a list of saved images.
        Params:
            obj: tensor or numpy array of shape (batch_size, channels, height, width)
            file_name: path and beggingin of filename to save.
                Images will be saved as `file_name_{image_number}.png`
    """
    img = convert_to_images(obj)

    for i, out in enumerate(img):
        current_file_name = file_name + '_%d.png' % i
        logger.info("Saving image to {}".format(current_file_name))
        out.save(current_file_name, 'png')


In [ ]:
# prepare interpolation data 
num_interps = 20 
truncation = 0.3

y_v1 = one_hot_from_names(['rhinoceros beetle'], batch_size=1)
y_v2 = one_hot_from_names(['roundabout'], batch_size=1)

z_v1 = truncated_noise_sample(truncation=truncation, batch_size=1, seed=3213)
z_v2 = truncated_noise_sample(truncation=truncation, batch_size=1, seed=1)

#create interpolation for both the category and the z vector 
interps_z = interpolate_hypersphere(z_v1 , z_v2 , num_interps)
interps_y = interpolate_hypersphere(y_v1,  y_v2, num_interps)

interps_z = interps_z.reshape(num_interps, z_dim)
interps_y = interps_y.reshape(num_interps, n_classes)

interps_z = torch.from_numpy(interps_z)
interps_y = torch.from_numpy(interps_y)

interps_z = interps_z.to('cuda')
interps_y = interps_y.to('cuda')

In [ ]:
# generated interpolated images
with torch.no_grad():
    output = model(interps_z, interps_y, truncation)
    save_as_images(output)

In [ ]:
# preserve background

#In this expermient we try to change the background of an arbitrary image 
#while keeping the foreground the same. We want the foregournd image to stay stable 
#so we need a function that preserves the values near zero but at the same time change the values away from zero.

# vals near 0 = background features?
# vals away from 0 = foreground features?

truncation = 0.7

y = one_hot_from_names(['tiger beetle'], batch_size=2)

z = np.zeros((2,128))
z_tmp = truncated_noise_sample(truncation=truncation, batch_size=1, seed=123)
z[0] = z_tmp[0]
# use sin func to modify values, values close to 0 stay close to 0, values away from 0 will be changed
z[1] = np.sin(z_tmp[0])
z = z.astype(np.float32)

y = torch.from_numpy(y)
z = torch.from_numpy(z)

y = y.to('cuda')
z = z.to('cuda')

with torch.no_grad():
   output = model(z, y, truncation)
   save_as_images(output)



In [ ]:
# zooming

# We try to zoom into a certain category to see the details of the generated images. 
# In order to that we need increase the weights of the vectors. 
# This doesn't work unless each feature in the vector space has either the value 1 or -1. 
# This is done by dividing each value in the vector by its absolute value namely $z/|z|$. 
# Then we can provide scaling by multipling by increasing negative values. 
# Note that this is highly dependent on the seed we are choosing. 
# Here we choose the seed = 2. 

torch.cuda.empty_cache()

truncation = .7

z_start = truncated_noise_sample(truncation=truncation, batch_size=1, seed=5432)

zoom_steps = 5
step_size = 0.2

y = one_hot_from_names(['tiger beetle'], batch_size=zoom_steps)

steps = np.arange(step_size,(zoom_steps*step_size)+step_size,step_size)
print(steps)
z = [-step*(z_start/np.abs(z_start)) for step in steps]
z = np.array(z).reshape(zoom_steps, z_dim)

y = torch.from_numpy(y)
z = torch.from_numpy(z)

y = y.to('cuda')
z = z.to('cuda')

In [ ]:
with torch.no_grad():
   output = model(z, y, truncation)
   save_as_images(output)

In [14]:
# breeding

# The idea is simple we just average the incoded labels and use the same seed vector. We use the combination 
# y^=ay1+(1−a)y2
torch.cuda.empty_cache()

truncation = 1.
a = 0.2

z = truncated_noise_sample(truncation=truncation, batch_size=1, seed=1)
y1 = one_hot_from_names(['albatross'], batch_size=1)
y2 = one_hot_from_names(['pelican'], batch_size=1)
y3 = a*y1+(1-a)*y2

z = np.array([z,z,z]).reshape(3, 128)
y = np.array([y1,y2,y3]).reshape(3, 1000)

y = torch.from_numpy(y)
z = torch.from_numpy(z)

y = y.to('cuda')
z = z.to('cuda')

In [15]:
with torch.no_grad():
   output = model(z, y, truncation)
   save_as_images(output)

INFO:__main__:Saving image to output_0.png
INFO:__main__:Saving image to output_1.png
INFO:__main__:Saving image to output_2.png
